# Preprocessing and clustering genes

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import igraph as ig
import leidenalg

In [3]:
# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

/home/ajararweh/miniconda3/envs/deepV_a100/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ajararweh/miniconda3/envs/deepV_a100/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ajararweh/miniconda3/envs/deepV_a100/lib/python3

scanpy==1.9.5 anndata==0.9.2 umap==0.5.3 numpy==1.24.4 scipy==1.11.1 pandas==2.0.3 scikit-learn==1.3.0 statsmodels==0.14.0 igraph==0.9.10 pynndescent==0.5.10


/home/ajararweh/miniconda3/envs/deepV_a100/lib/python3.10/site-packages/umap/umap_.py:660: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


In [9]:
import json

# Specify the JSON file path
json_file_path = "data/gene_embeddings.json"


with open(json_file_path, "r") as json_file:
    json_data = json.load(json_file)

16040


In [4]:
results_file = 'genellm_funmi.h5ad'  # the file that will store the analysis results

In [81]:
adata = sc.read_text("gene_embeds.tsv", delimiter='\t', first_column_names=1, dtype='float32').transpose()

In [82]:
adata.var_names_make_unique() 

Identify highly-variable genes.

In [83]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

Scale each gene to unit variance.

In [84]:
sc.pp.scale(adata, max_value=10)

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

In [85]:
sc.tl.pca(adata, svd_solver='arpack')

In [86]:
adata.write(results_file)

In [87]:
adata

## Computing the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

In [88]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

## Embedding the neighborhood graph

In [89]:
sc.tl.umap(adata)
sc.tl.tsne(adata)

In [94]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP']) #normalized, logarithmized, but uncorrected gene expression

In [93]:
sc.pl.umap(adata, color=['CST3'], use_raw=False)##normalized, logarithmized, scaled gene expression

In [92]:
sc.pl.tsne(adata, color=['CST3', 'NKG7', 'PPBP'])

## Clustering the neighborhood graph

In [77]:
sc.tl.leiden(adata)

Plot the clusters, which agree quite well with the result of Seurat.

In [91]:
sc.pl.umap(adata, color=['leiden', 'CST3', 'FAM13B', 'DND1'])

Save the result.

In [33]:
adata.write(results_file)

In [48]:
new_cluster_names = [
    "Metabotropic glutamate/pheromone receptors","rRNA modification","Signaling by Interleukins","DNA Repair","4","Translation","Keratan sulfate degradation","7",
"Keratinization","9","10","Keratinization","12","G alpha(s) signalling events"]
adata.rename_categories('leiden', new_cluster_names)

In [90]:
sc.pl.umap(adata, color='leiden', legend_loc='on data', title='', frameon=False, save='.pdf')

In [95]:
adata

In [53]:
# `compression='gzip'` saves disk space, but slows down writing and subsequent reading
adata.write(results_file, compression='gzip')  

If you want to export to "csv", you have the following options:

In [55]:
# Export single fields of the annotation of observations
# adata.obs[['n_counts', 'louvain_groups']].to_csv(
#     './write/pbmc3k_corrected_louvain_groups.csv')

# Export single columns of the multidimensional annotation
# adata.obsm.to_df()[['X_pca1', 'X_pca2']].to_csv(
#     './write/pbmc3k_corrected_X_pca.csv')

# Or export everything except the data using `.write_csvs`.
# Set `skip_data=False` if you also want to export the data.
# adata.write_csvs(results_file[:-5], )